In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import os

In [2]:
data = pd.read_csv('data/train/trainx.txt', sep='\t')
features = data.columns.values
print("特征如下：{} \n".format(features))
print("数据数目: ", len(data))

特征如下：['ITEM_NAME' 'TYPE'] 

数据数目:  499447


## 数据清洗
* 替换一些字符 类似于 （ ） ‘ ’ _

In [3]:
import re
from zhon.hanzi import punctuation
from string import punctuation as english_punc
def  rm_punc(strs):
    return re.sub(r"[{}]+".format(punctuation + english_punc)," ",strs)
data[features[0]]  = data[features[0]].apply(rm_punc)
data.head()

,ITEM_NAME,TYPE
0,腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费,本地生活--游戏充值--QQ充值
1,腾讯q币充值30元qq币30qb30q币自动充值,本地生活--游戏充值--QQ充值
2,腾讯QQ红钻包月卡 QQ红钻一个月QQ红钻1个月,本地生活--游戏充值--QQ充值
3,腾讯QQ蓝钻贵族31个月直充,本地生活--游戏充值--QQ充值
4,腾讯QQ币148元148QQ币148个直充148Q币148个Q币148个QQB★自动充值,本地生活--游戏充值--QQ充值


In [4]:
# 因为有的label里本身就存在空格，所以先要把空格去掉，再去掉- - ， 再以空格划分
data = shuffle(data)
data_y = data[features[1]]

# 取训练数据和测试数据
train = data.sample(frac= 0.8).reset_index()
test = data.sample(frac= 0.2).reset_index()


print("训练数据数目:  ", len(train))
print("测试数据数目:  ", len(test))

train_x  = train[features[0]]
train_y  = train[features[1]]

test_x  = test[features[0]]
test_y  = test[features[1]]

print(train_x.shape, test_x.shape)

# 分出数据集 训练集 测试集 数据集用来训练词向量
data_y = data_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
data_y= [each.split() for each in data_y]

train_y = train_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
train_y = [each.split() for each in train_y]

test_y = test_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
test_y = [each.split() for each in test_y]

训练数据数目:   399558
测试数据数目:   99889
(399558,) (99889,)


## 分词

In [5]:
import jieba
from random import shuffle
def participle(data, doshuffle = False):
    words = []
    for i in range(len(data)):
        result=[]
        seg_list = jieba.cut(data.iloc[i])
        for w in seg_list :#读取每一行分词
            if w != " ":
                result.append(w)
        # 打乱每条数据的分词
        if doshuffle:
            shuffle(result)
        words.append(result)#将该行分词写入列表形式的总分词列表
    return words

In [6]:
# 每条数据的分词结果是个长度不等的词列表
word_data = participle(data[features[0]], doshuffle = False)
word_data_train = participle(train_x,doshuffle = False)
word_data_test = participle(test_x, doshuffle = False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Richado\AppData\Local\Temp\jieba.cache
Loading model cost 0.689 seconds.
Prefix dict has been built succesfully.


In [7]:
data[features[0]][0]

'腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费'

In [8]:
from random import shuffle
word_data[8]

['kate',
 'spade',
 '凯特',
 '丝蓓',
 '纯色',
 '女士',
 '时尚',
 '单肩',
 '手提包',
 'WKRU3367',
 '浅蓝色',
 'WKRU3367',
 '417']

## 标签分离(一个标签分成三个标签)

In [9]:
train_y = pd.DataFrame(train_y, columns=['label1', 'label2', 'label3'])
test_y = pd.DataFrame(test_y, columns=['label1', 'label2', 'label3'])
data_y = pd.DataFrame(data_y, columns=['label1', 'label2', 'label3'])

In [10]:
len(data_y["label1"].unique())

22

In [11]:
len(data_y["label2"].unique())

191

In [12]:
len(data_y["label3"].unique())

1192

## 标签onn_hot

In [13]:
from keras.utils import to_categorical
# 先生成唯一数组
y_label1 = data_y['label1'].unique().tolist()
y_label2 = data_y['label2'].unique().tolist()
y_label3 = data_y['label3'].unique().tolist()

# 获取在唯一数组中的索引(训练集和测试集各有3个标签需要处理)
train_y_label1_map = train_y['label1'].apply(lambda x: y_label1.index(x))
train_y_label2_map = train_y['label2'].apply(lambda x: y_label2.index(x))
train_y_label3_map = train_y['label3'].apply(lambda x: y_label3.index(x))
test_y_label1_map = test_y['label1'].apply(lambda x: y_label1.index(x))
test_y_label2_map = test_y['label2'].apply(lambda x: y_label2.index(x))
test_y_label3_map = test_y['label3'].apply(lambda x: y_label3.index(x))

Using TensorFlow backend.


In [14]:
# 生成对应one-hot(用做训练模型的标签)
train_y_label1_one_hot = to_categorical(train_y_label1_map, 22)
train_y_label2_one_hot = to_categorical(train_y_label2_map, 191)
train_y_label3_one_hot = to_categorical(train_y_label3_map, 1192)
test_y_label1_one_hot = to_categorical(test_y_label1_map, 22)
test_y_label2_one_hot = to_categorical(test_y_label2_map, 191)
test_y_label3_one_hot = to_categorical(test_y_label3_map, 1192)

# 词向量训练

In [15]:
from gensim.models.word2vec import Word2Vec 
if os.path.exists("w2v_model"):
    model = Word2Vec.load("w2v_model")
    print(1)
else:
    sentences = word_data
    model= Word2Vec(size=50, window=10, min_count = 1)
    model.build_vocab(sentences)
    model.train(sentences,total_examples = model.corpus_count,epochs = model.iter)
    model.save("w2v_model")

1


## 填充词向量(构造输入输出)（让每个句子拥有同样的维度）

In [16]:
from keras.preprocessing import sequence
x_train = [model[word] for word in word_data_train]
x_test = [model[word] for word in word_data_test]
x_train = sequence.pad_sequences(x_train, maxlen=18)
x_test  = sequence.pad_sequences(x_test, maxlen=18) 

c:\python36\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
c:\python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


## 默认参数

In [17]:
maxlen = 10 # 一句话的填充的词的个数
batch_size = 32 
embedding_dims = 50 # 一个词的词向量长度
epochs = 100
is_training = True


## TextCnn

In [18]:
import tensorflow as tf
import numpy as np


class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, maxlen, sequence_length, num_classes, filter_sizes, embedding_size, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.float32, [None, sequence_length, embedding_size], name="input_x")
        self.input_y1 = tf.placeholder(tf.float32, [None, num_classes[0]], name="input_y1")
        self.input_y2 = tf.placeholder(tf.float32, [None, num_classes[1]], name="input_y2")
        self.input_y3 = tf.placeholder(tf.float32, [None, num_classes[2]], name="input_y3")
        
        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)
        
        pooled_outputs = []
        self.input_x_expand = tf.expand_dims(self.input_x, -1)
        print(self.input_x_expand.shape)
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                print(W.shape)
                conv = tf.nn.conv2d(
                    self.input_x_expand,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        self.h_drop = self.h_pool_flat
        
        # Final (unnormalized) scores and predictions
        
        with tf.name_scope("output1"):
            W1 = tf.get_variable(
                "W11",
                shape=[num_filters_total, 64],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[64]), name="b11")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output1")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            
            W2 = tf.get_variable(
                "W12",
                shape=[64, num_classes[0]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[0]]), name="b12")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores1 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions1 = tf.argmax(self.scores1, 1, name="predictions")
            
        with tf.name_scope("output2"):
            W1 = tf.get_variable(
                "W21",
                shape=[num_filters_total, 128],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[128]), name="b21")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output2")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            
            W2 = tf.get_variable(
                "W22",
                shape=[128, num_classes[1]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[1]]), name="b22")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores2 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions2 = tf.argmax(self.scores2, 1, name="predictions")
            
        with tf.name_scope("output3"):
            W1 = tf.get_variable(
                "W31",
                shape=[num_filters_total, 256],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[256]), name="b31")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output3")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            
            W2 = tf.get_variable(
                "W32",
                shape=[256, num_classes[2]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[2]]), name="b32")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores3 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions3 = tf.argmax(self.scores3, 1, name="predictions")
        # Calculate mean cross-entropy loss
        
        with tf.name_scope("loss"):
            losses1 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores1, labels=self.input_y1)
            losses2 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores2, labels=self.input_y2)
            losses3 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores3, labels=self.input_y3)
            self.loss = 0.1 * tf.reduce_mean(losses1) + 0.2 * tf.reduce_mean(losses2) + 0.7*tf.reduce_mean(losses3) + l2_reg_lambda * l2_loss
            tf.summary.scalar('loss1',tf.reduce_mean(losses1))
            tf.summary.scalar('loss2',tf.reduce_mean(losses2))
            tf.summary.scalar('loss3',tf.reduce_mean(losses3))
            tf.summary.scalar('loss',self.loss)
        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions1 = tf.equal(self.predictions1, tf.argmax(self.input_y1, 1))
            correct_predictions2 = tf.equal(self.predictions2, tf.argmax(self.input_y2, 1))
            correct_predictions3 = tf.equal(self.predictions3, tf.argmax(self.input_y3, 1))
#           单独的准确率
            self.accuracy1 = tf.reduce_mean(tf.cast(correct_predictions1, "float"), name="accuracy1")
            self.accuracy2 = tf.reduce_mean(tf.cast(correct_predictions2, "float"), name="accuracy2")
            self.accuracy3 = tf.reduce_mean(tf.cast(correct_predictions3, "float"), name="accuracy3")
#           一起的准确率
            self.acc = tf.reduce_mean(tf.cast(correct_predictions1, "float")*tf.cast(correct_predictions2, "float")*tf.cast(correct_predictions3, "float"))
            tf.summary.scalar('acc1',self.accuracy1)
            tf.summary.scalar('acc2',self.accuracy2)
            tf.summary.scalar('acc3',self.accuracy3)
            tf.summary.scalar('acc',self.acc)

In [ ]:
import math
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
  
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = np.min(((batch_num + 1) * batch_size, data_size))
            yield shuffled_data[start_index:end_index]
            

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from tensorflow.contrib import learn

# 特征数这里是第三级标签的特征数
max_features = 1192

# 句子填充的长度
maxlen = 16

batch_size = 32

# 词向量长度
embedding_dims = 50

filters = 64
kernel_size = 5
hidden_dims = 128
epochs = 100

# Parameters
# ==================================================

# Data loading params
# tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
# tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
# tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
# tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
# tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
# tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
# tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
# tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
# tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
# tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
# tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
# tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
# tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
# tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
# tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS

def train(x_train, y_train, x_dev, y_dev):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
#           allow_soft_placement=FLAGS.allow_soft_placement,
#           log_device_placement=FLAGS.log_device_placement
        )
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(maxlen = 10, sequence_length = 18, num_classes = [22, 191, 1192], filter_sizes = [1,2, 3, 4, 5], embedding_size = 50, num_filters = 64)

            # Define Training procedure

            global_step = tf.Variable(0, name="global_step", trainable=False)
#           学习率衰减
            learning_rate = tf.train.exponential_decay(
                0.01,
                global_step,
                x_train.shape[0] / 32,
                0.99,
                staircase=True,)
            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Initialize all variables
            sess.run(tf.global_variables_initializer())
            
            merged = tf.summary.merge_all()
            
            writer = tf.summary.FileWriter("fwwb",sess.graph)
            acc_sum = np.array([0.0,0.0,0.0,0.0])
            loss_sum = 0
            
            def train_step(x_batch, y_batch1, y_batch2, y_batch3, loss_sum, acc_sum, interal = 10):
  
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y1: y_batch1,
                  cnn.input_y2: y_batch2,
                  cnn.input_y3: y_batch3
                }
#         三个标签单独的准确率， 一起的准确率共四个准确率
                rs, _, step,  loss, accuracy1, accuracy2, accuracy3, acc = sess.run(
                    [merged, train_op, global_step, cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                acc_sum += [accuracy1, accuracy2, accuracy3, acc]
                loss_sum += loss
                if step % interal == 0:
                    print("{}: step {}, loss {:g}, acc {}".format(time_str, step, loss_sum/interal, acc_sum/interal))
                    loss_sum = 0
                    acc_sum = np.array([0.0,0.0,0.0,0.0])
                    writer.add_summary(rs, step)
                return loss_sum, acc_sum
            # 评估步骤
            def dev_step(x_batch, y_batch):
                """
                Evaluates model on a dev set
                """

                dev_batches = batch_iter(
                    list(zip(x_batch, y_batch[0], y_batch[1], y_batch[2])), 640, 1)
                step = 0
                loss = 0
                accuracy = np.array([0.0, 0.0, 0.0,0.0])
                for batch in dev_batches:
                    x_batch_dev, y_batch1_dev, y_batch2_dev, y_batch3_dev = zip(*batch)
                    feed_dict = {
                      cnn.input_x: x_batch_dev,
                      cnn.input_y1: y_batch1_dev,
                      cnn.input_y2: y_batch2_dev,
                      cnn.input_y3: y_batch3_dev
                    }
                    step,  temp_loss, temp_accuracy1, temp_accuracy2, temp_accuracy3 , acc= sess.run(
                        [global_step,cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                        feed_dict)
                    accuracy[0] += temp_accuracy1 * len(x_batch_dev)
                    accuracy[1] += temp_accuracy2 * len(x_batch_dev)
                    accuracy[2] += temp_accuracy3 * len(x_batch_dev)
                    accuracy[3] += acc * len(x_batch_dev)
                    loss += temp_loss * len(x_batch_dev)
                accuracy /= x_batch.shape[0]
                loss /= x_batch.shape[0]
                time_str = datetime.datetime.now().isoformat()                    

                print("{}: step {}, loss {:g}, acc {} ".format(time_str, step, loss, accuracy))


            # Generate batches
            batches = batch_iter(
                list(zip(x_train, y_train[0],y_train[1], y_train[2] )), 64, 200)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch1, y_batch2, y_batch3 = zip(*batch)
                loss_sum, acc_sum = train_step(x_batch, y_batch1, y_batch2, y_batch3,  loss_sum, acc_sum, 40)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 200 == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev)
                    print("")

In [ ]:
y_train = [train_y_label1_one_hot,train_y_label2_one_hot,train_y_label3_one_hot]
y_test = [test_y_label1_one_hot,test_y_label2_one_hot,test_y_label3_one_hot]
train(x_train, y_train,  x_test, y_test)



(?, 18, 50, 1)
(1, 50, 1, 64)
(2, 50, 1, 64)
(3, 50, 1, 64)
(4, 50, 1, 64)
(5, 50, 1, 64)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

2019-02-21T21:01:53.784666: step 40, loss 3.47771, acc [0.72382813 0.52539062 0.35898438 0.30742188]
2019-02-21T21:01:54.401167: step 80, loss 2.37762, acc [0.81289062 0.65429688 0.515625   0.46679688]
2019-02-21T21:01:55.118245: step 120, loss 2.04899, acc [0.81054688 0.69335938 0.56367188 0.50546875]
2019-02-21T21:01:55.838324: step 160, loss 1.86156, acc [0.82148438 0.70507812 0.57539063 0.5203125 ]
2019-02-21T21:01:56.387035: step 200, loss 1.72702, acc [0.8390625  0.72460938 0.5984375  0.54609375]

Evaluation:
2019-02-21T21:02:01.492231: step 200, loss 1.69683, acc [0.83877104 0.72165103 0.59558109 0.54383366] 

2019-02-21T21:02:02.323780: step 240, loss 1.70866, acc [0.83085937 0.71679688 0.57773438 0.5

2019-02-21T21:03:06.576014: step 2520, loss 1.19513, acc [0.8625    0.7765625 0.6890625 0.628125 ]
2019-02-21T21:03:07.118990: step 2560, loss 1.09461, acc [0.8796875  0.79804688 0.70351562 0.65585938]
2019-02-21T21:03:07.641334: step 2600, loss 1.11923, acc [0.87578125 0.79179687 0.69765625 0.63984375]

Evaluation:
2019-02-21T21:03:10.337990: step 2600, loss 1.08665, acc [0.87336944 0.79284005 0.70357096 0.65155323] 

2019-02-21T21:03:10.925779: step 2640, loss 1.06311, acc [0.87421875 0.79492188 0.71054688 0.65039062]
2019-02-21T21:03:11.439085: step 2680, loss 1.17053, acc [0.86484375 0.77929688 0.68867188 0.634375  ]
2019-02-21T21:03:11.937070: step 2720, loss 1.21671, acc [0.86328125 0.7765625  0.67148438 0.61796875]
2019-02-21T21:03:12.556773: step 2760, loss 1.17344, acc [0.86953125 0.77773437 0.69257813 0.6453125 ]
2019-02-21T21:03:13.074724: step 2800, loss 1.11666, acc [0.86640625 0.7890625  0.70078125 0.646875  ]

Evaluation:
2019-02-21T21:03:15.769708: step 2800, loss 1.063

2019-02-21T21:04:19.563561: step 5120, loss 1.05557, acc [0.88085938 0.80078125 0.71367187 0.66445312]
2019-02-21T21:04:20.020926: step 5160, loss 1.0913, acc [0.87226563 0.77890625 0.7046875  0.64570313]
2019-02-21T21:04:20.496419: step 5200, loss 1.04471, acc [0.87539062 0.78984375 0.70742187 0.64179688]

Evaluation:
2019-02-21T21:04:23.391317: step 5200, loss 0.971603, acc [0.88278989 0.80781668 0.7286488  0.67359769] 

2019-02-21T21:04:24.013648: step 5240, loss 1.10372, acc [0.87070313 0.79335937 0.70234375 0.64609375]
2019-02-21T21:04:24.503351: step 5280, loss 1.02763, acc [0.87734375 0.80664062 0.71679688 0.6703125 ]
2019-02-21T21:04:25.016872: step 5320, loss 1.15065, acc [0.86953125 0.78242188 0.69960937 0.63789063]
2019-02-21T21:04:25.493565: step 5360, loss 1.0586, acc [0.88320312 0.8015625  0.71914062 0.66796875]
2019-02-21T21:04:25.998228: step 5400, loss 1.10055, acc [0.87929687 0.79804688 0.709375   0.6578125 ]

Evaluation:
2019-02-21T21:04:29.118228: step 5400, loss 0.

2019-02-21T21:05:28.461704: step 7720, loss 0.950908, acc [0.88828125 0.80585938 0.73164063 0.67304688]
2019-02-21T21:05:28.899535: step 7760, loss 1.02923, acc [0.884375   0.8        0.715625   0.66171875]
2019-02-21T21:05:29.340420: step 7800, loss 0.912708, acc [0.8921875  0.81484375 0.740625   0.684375  ]

Evaluation:
2019-02-21T21:05:32.414894: step 7800, loss 0.929728, acc [0.88624373 0.81438397 0.74059206 0.68660213] 

2019-02-21T21:05:32.960761: step 7840, loss 1.03328, acc [0.88046875 0.80195313 0.72265625 0.66796875]
2019-02-21T21:05:33.419542: step 7880, loss 0.967098, acc [0.884375   0.80234375 0.7296875  0.66875   ]
2019-02-21T21:05:33.865819: step 7920, loss 0.993976, acc [0.88046875 0.81054688 0.71640625 0.6640625 ]
2019-02-21T21:05:34.310838: step 7960, loss 1.00625, acc [0.8765625  0.80820313 0.73085937 0.67304688]
2019-02-21T21:05:34.763631: step 8000, loss 1.0196, acc [0.8796875  0.7984375  0.71523437 0.66132813]

Evaluation:
2019-02-21T21:05:37.849720: step 8000, lo

2019-02-21T21:06:34.165417: step 10320, loss 0.958911, acc [0.884375   0.80859375 0.73789063 0.68085938]
2019-02-21T21:06:34.595826: step 10360, loss 1.04466, acc [0.87304688 0.79882812 0.71875    0.659375  ]
2019-02-21T21:06:35.027080: step 10400, loss 0.994881, acc [0.87070313 0.80390625 0.71953125 0.66679687]

Evaluation:
2019-02-21T21:06:37.727981: step 10400, loss 0.897438, acc [0.88942727 0.81744737 0.74742965 0.68946531] 

2019-02-21T21:06:38.243683: step 10440, loss 0.979976, acc [0.88828125 0.81953125 0.73515625 0.68359375]
2019-02-21T21:06:38.681081: step 10480, loss 0.967614, acc [0.88828125 0.809375   0.74453125 0.69179687]
2019-02-21T21:06:39.102947: step 10520, loss 1.00284, acc [0.87695312 0.80429688 0.72890625 0.67109375]
2019-02-21T21:06:39.533049: step 10560, loss 0.976637, acc [0.88632813 0.80976563 0.72734375 0.67460937]
2019-02-21T21:06:39.970444: step 10600, loss 0.930398, acc [0.884375   0.8140625  0.73867187 0.67734375]

Evaluation:
2019-02-21T21:06:42.681357: s

...

2019-02-21T23:16:10.349471: step 328160, loss 0.671653, acc [0.915625   0.8625     0.80234375 0.740625  ]
2019-02-21T23:16:10.770622: step 328200, loss 0.648851, acc [0.9140625  0.84960938 0.8140625  0.73671875]

Evaluation:
2019-02-21T23:16:13.489099: step 328200, loss 0.707009, acc [0.91300343 0.86020483 0.80912813 0.74623833] 

2019-02-21T23:16:13.993006: step 328240, loss 0.697756, acc [0.91054687 0.84023437 0.8078125  0.73398438]
2019-02-21T23:16:14.428807: step 328280, loss 0.678515, acc [0.91601562 0.86757812 0.79960937 0.73867187]
2019-02-21T23:16:14.834074: step 328320, loss 0.698696, acc [0.90625    0.84765625 0.79023438 0.7234375 ]
2019-02-21T23:16:15.249306: step 328360, loss 0.689148, acc [0.90898437 0.84492188 0.78476563 0.72148437]
2019-02-21T23:16:15.678056: step 328400, loss 0.757612, acc [0.90703125 0.85273438 0.78085938 0.72109375]

Evaluation:
2019-02-21T23:16:18.410675: step 328400, loss 0.706553, acc [0.91265304 0.8607254  0.80696573 0.74430618] 

2019-02-21T23:16

2019-02-21T23:17:18.351511: step 330680, loss 0.65684, acc [0.92148438 0.85703125 0.80351562 0.74140625]
2019-02-21T23:17:18.780364: step 330720, loss 0.671605, acc [0.9109375  0.8515625  0.80429688 0.73632812]
2019-02-21T23:17:19.208220: step 330760, loss 0.622263, acc [0.91445312 0.86875    0.82265625 0.75273437]
2019-02-21T23:17:19.644054: step 330800, loss 0.648811, acc [0.903125   0.8484375  0.81367188 0.7421875 ]

Evaluation:
2019-02-21T23:17:22.495429: step 330800, loss 0.699251, acc [0.91411467 0.86231717 0.81032946 0.74918159] 

2019-02-21T23:17:23.018029: step 330840, loss 0.667861, acc [0.89765625 0.85859375 0.79140625 0.72851562]
2019-02-21T23:17:23.443891: step 330880, loss 0.655882, acc [0.91601562 0.86289063 0.81132812 0.7421875 ]
2019-02-21T23:17:23.875736: step 330920, loss 0.623852, acc [0.91328125 0.859375   0.8125     0.74804688]
2019-02-21T23:17:24.459174: step 330960, loss 0.626085, acc [0.91640625 0.86210937 0.79973958 0.73567708]
2019-02-21T23:17:24.896007: step

2019-02-21T23:18:22.599135: step 333200, loss 0.69895, acc [0.9129734  0.86133608 0.81114037 0.74739961] 

2019-02-21T23:18:23.096103: step 333240, loss 0.674871, acc [0.90507812 0.85703125 0.79921875 0.73554688]
2019-02-21T23:18:23.506572: step 333280, loss 0.720491, acc [0.90976563 0.84726563 0.79257813 0.7265625 ]
2019-02-21T23:18:23.912726: step 333320, loss 0.683434, acc [0.90859375 0.8578125  0.79492188 0.73085937]
2019-02-21T23:18:24.322238: step 333360, loss 0.659302, acc [0.91640625 0.86445313 0.80820313 0.73671875]
2019-02-21T23:18:24.744013: step 333400, loss 0.681696, acc [0.90859375 0.85664063 0.79609375 0.72890625]

Evaluation:
2019-02-21T23:18:27.607307: step 333400, loss 0.704434, acc [0.91352401 0.86242729 0.80901801 0.7470292 ] 

2019-02-21T23:18:28.091240: step 333440, loss 0.682922, acc [0.90742188 0.85625    0.79335937 0.72890625]
2019-02-21T23:18:28.503421: step 333480, loss 0.664053, acc [0.91367188 0.85546875 0.80234375 0.73164063]
2019-02-21T23:18:28.909581: st

2019-02-21T23:19:25.552020: step 335760, loss 0.69658, acc [0.90234375 0.84453125 0.80429688 0.734375  ]
2019-02-21T23:19:25.973795: step 335800, loss 0.670861, acc [0.90742188 0.85429687 0.80117187 0.73359375]

Evaluation:
2019-02-21T23:19:28.713056: step 335800, loss 0.696063, acc [0.91375427 0.86049515 0.81142068 0.74806035] 

2019-02-21T23:19:29.246782: step 335840, loss 0.681403, acc [0.90507812 0.85546875 0.7875     0.71875   ]
2019-02-21T23:19:29.674211: step 335880, loss 0.62416, acc [0.92617187 0.8640625  0.80390625 0.74296875]
2019-02-21T23:19:30.090638: step 335920, loss 0.671852, acc [0.91367188 0.85625    0.79804688 0.7359375 ]
2019-02-21T23:19:30.543022: step 335960, loss 0.67465, acc [0.921875   0.85859375 0.796875   0.73710937]
2019-02-21T23:19:30.998801: step 336000, loss 0.722762, acc [0.90820312 0.84804687 0.80078125 0.73945313]

Evaluation:
2019-02-21T23:19:33.888663: step 336000, loss 0.704543, acc [0.91382434 0.86132607 0.80834726 0.74641852] 

2019-02-21T23:19:34

2019-02-21T23:20:33.184111: step 338280, loss 0.687568, acc [0.89804688 0.85703125 0.79960937 0.72734375]
2019-02-21T23:20:33.608298: step 338320, loss 0.665811, acc [0.90859375 0.85898438 0.80625    0.73828125]
2019-02-21T23:20:34.066332: step 338360, loss 0.640547, acc [0.91953125 0.8625     0.8015625  0.73828125]
2019-02-21T23:20:34.501169: step 338400, loss 0.616488, acc [0.92148438 0.8625     0.82148438 0.76015625]

Evaluation:
2019-02-21T23:20:37.469512: step 338400, loss 0.697024, acc [0.91366417 0.86216701 0.81092012 0.74909149] 

2019-02-21T23:20:38.015034: step 338440, loss 0.646262, acc [0.91523438 0.86523438 0.79804688 0.73476562]
2019-02-21T23:20:38.464830: step 338480, loss 0.683138, acc [0.896875   0.85039062 0.79726562 0.71835938]
2019-02-21T23:20:38.917619: step 338520, loss 0.622524, acc [0.91132813 0.86679688 0.82148438 0.7578125 ]
2019-02-21T23:20:39.373400: step 338560, loss 0.620372, acc [0.92070312 0.86484375 0.81484375 0.74882812]
2019-02-21T23:20:39.795825: ste

In [ ]:
test_y_label3_one_hot.shape

In [ ]:
train_y_label3_one_hot.shape